<a href="https://colab.research.google.com/github/lefko2000/lefko2000.github.io/blob/master/recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import os
import random
from zipfile import ZipFile

from google.colab import drive
import tensorflow as tf
import numpy as np


## Data loading

In [ ]:
drive.mount("/content/gdrive")
my_path = "/content/gdrive/path/to/zip_archive"

with ZipFile(os.path.join(my_path, "recommendation.zip")) as myzip:
    myzip.extractall()

In [3]:
records_dir = os.path.join(my_path, "recommendation/dataset")
records_paths = [os.path.join(records_dir, fname) for fname in os.listdir(records_dir)]

dataset = tf.data.TFRecordDataset(filenames = records_paths)

In [4]:
with open(os.path.join(my_path, "recommendation/movie_title_by_index.json"))as f:
  movie_title_by_index = json.load(f)

movie_count = len(movie_title_by_index)

In [5]:
SCHEMA = {
        "userIndex": tf.io.FixedLenFeature([], tf.int64),
        "movieIndices": tf.io.RaggedFeature(tf.int64, row_splits_dtype=tf.int64),
        "timestamps": tf.io.RaggedFeature(tf.int64, row_splits_dtype=tf.int64)
}

def parse_movie_indices(row):
  parsed_row = tf.io.parse_example(row, SCHEMA)
  return parsed_row["movieIndices"]

for row in dataset.take(1):
  exemple = parse_movie_indices(row)
  print(exemple.numpy())

[22  6  5 25 12  3 26 14 34 21  1  4 32 28 11 15 30 18 23 10 24 36 17 19
 35  7 31 27 16  9 20  2  8 37 33 38 13  0 29]


In [6]:
row_count = dataset.reduce(0, lambda x, _ : x + 1).numpy()
print("row count:", row_count)

row count: 324849


## Model construction

In [7]:
max_len = 10

class ReversibleEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dims):
        super().__init__()
        self.w = tf.Variable(tf.random.normal([vocab_size, embedding_dims]))

    def call(self, x, reversed=False):
        if reversed:
            return tf.matmul(x, self.w, transpose_b=True)
        return tf.gather(self.w, x)

class BiasLayer(tf.keras.layers.Layer):
    def __init__(self, size):
      super().__init__()
      self.bias = tf.Variable(tf.zeros([size]))

    def call(self, x):
      return tf.add(x, self.bias)

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation= tf.nn.gelu), tf.keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = ReversibleEmbedding(vocab_size, embed_dim)
        self.pos_emb = tf.keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

movie_seq = tf.keras.layers.Input(shape=(None, ), dtype="int32")
embedding_layer = TokenAndPositionEmbedding(max_len, movie_count + 2, embed_dim)
x = embedding_layer(movie_seq)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = tf.keras.layers.Dense(embed_dim, activation= tf.nn.gelu)(x)
x = embedding_layer.token_emb(x, reversed=True) #allow shared weight with input embedding
last_bias = BiasLayer(movie_count + 2)
x = last_bias(x)
soft_max = tf.keras.layers.Softmax()
x = soft_max(x)

model = tf.keras.Model(inputs=movie_seq, outputs=x)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 token_and_position_embeddin  (None, None, 32)         1307840   
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_block (Transfor  (None, None, 32)         10656     
 merBlock)                                                       
                                                                 
 dense_2 (Dense)             (None, None, 32)          1056      
                                                                 
 reversible_embedding (Rever  (None, None, 40860)      1307520   
 sibleEmbedding)                                             

### Data set preprocessing

In [9]:
row_repeat_count = 2
mask_rate = 0.1
batch_size = 64
seed = 1234
train_share = 0.5
valid_share = 0.25

distinct_movie_ids = [int(k) for k in movie_title_by_index.keys()][:100]
mask_token = max(distinct_movie_ids) + 1
vocab = distinct_movie_ids + [mask_token]
integer_lookup = tf.keras.layers.IntegerLookup(vocabulary=vocab)
mask_token_id = integer_lookup.get_vocabulary()[mask_token]

def get_and_apply_mask(movies, mask_token_id):
    draws =  tf.map_fn(lambda x: tf.random.uniform([]), movies, fn_output_signature=tf.float32)
    mask = draws < mask_rate
    x = tf.where(mask, tf.constant(mask_token_id, dtype=tf.int64), movies)
    y = tf.where(mask, movies, tf.constant(mask_token_id, dtype=tf.int64))
    return x, y

dataset_train = dataset.shuffle(1000, seed=seed).take(int(row_count * train_share))
dataset_valid = dataset.shuffle(1000, seed=seed).skip(int(row_count * train_share)).take(int(row_count * valid_share))
dataset_test = dataset.shuffle(1000, seed=seed).skip(int(row_count * (train_share + valid_share)))

def proprocess_dataset(dataset):
    return (dataset
        .map(parse_movie_indices)
        .map(integer_lookup)
        .map(lambda x: x[-max_len:])
        .repeat(row_repeat_count)
        .map(lambda movies: get_and_apply_mask(movies, mask_token))
        .cache()
        .padded_batch(32, padding_values=mask_token_id)
        .prefetch(tf.data.AUTOTUNE)
)

preprocessed_train = proprocess_dataset(dataset_train)
preprocessed_valid = proprocess_dataset(dataset_valid)

In [10]:
def masked_categorical_cross_entropy(y_true, y_pred):
    mask = y_true != mask_token_id
    true_probas = tf.gather(y_pred, y_true, axis=-1)
    mask_count = tf.reduce_sum(tf.cast(mask, tf.float32))
    sum_of_log_probs =  -tf.reduce_sum(tf.where(mask, tf.math.log(true_probas), tf.constant(0, dtype=tf.float32)))
    return sum_of_log_probs / mask_count

# def top_10_rate(y_true, y_pred):
#     mask = y_true != mask_token_id
#     values, indices  = tf.math.top_k(y_pred, k=10)
#     in_top_10 = tf.reduce_sum(tf.cast(tf.math.equal(tf.cast(y_true, tf.int32), tf.transpose(indices, [2, 0, 1])), tf.float32), axis=0)
#     mask_count = tf.reduce_sum(tf.cast(mask, tf.float32))
#     in_top_10_masked_sum =  tf.reduce_sum(tf.where(mask, in_top_10, tf.constant(0, dtype=tf.float32)))
#     return in_top_10_masked_sum / mask_count


model.compile(optimizer='adam',
              loss = masked_categorical_cross_entropy,
            #   metrics = [top_10_rate],
)

history = model.fit(
    x=preprocessed_train.take(1000),
    validation_data=preprocessed_valid.take(1000),
    epochs=5
)

Epoch 1/5
1000/1000 [==============================] - 38s 36ms/step - loss: 282.9026 - val_loss: 142.3593
Epoch 2/5
1000/1000 [==============================] - 36s 36ms/step - loss: 135.3023 - val_loss: 128.9986
Epoch 3/5
1000/1000 [==============================] - 41s 41ms/step - loss: 130.3708 - val_loss: 126.8894
Epoch 4/5
1000/1000 [==============================] - 41s 41ms/step - loss: 128.7454 - val_loss: 127.7535
Epoch 5/5
1000/1000 [==============================] - 36s 36ms/step - loss: 128.2688 - val_loss: 127.8793
